### 1. XGBoost的损失函数必须为二阶可导的凸函数
- 海塞矩阵必为正定矩阵，因为XGB需要计算二阶偏导数的累计平方和

In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification, make_regression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import roc_auc_score, f1_score, mean_squared_error, accuracy_score, recall_score
from xgboost import XGBClassifier, XGBRegressor

X, y = make_classification(n_samples=1000, n_features=10, n_informative=8, n_classes=2, random_state=2018)
X_train_clf, X_test_clf, y_train_clf, y_test_clf = train_test_split(X, y, test_size=0.25, random_state=2018)

X, y = make_regression(n_samples=1000, n_features=10, n_informative=8, n_targets=1, random_state=2018)
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(X, y, test_size=0.25, random_state=2018)

### 1.以ln(cosh(x))为损失函数
https://baike.baidu.com/item/cosh/3535060

In [2]:
# 定义函数
def log_cosh_obj(y_true, y_pred):
    delta = y_pred - y_true 
    grad = np.tanh(delta)
    hess = (1.0 - grad*grad)
    return grad, hess

# 回归问题
model = XGBRegressor(objective=log_cosh_obj)
model.fit(X_train_reg, y_train_reg)
y_pred = model.predict(X_test_reg)
mse = mean_squared_error(y_test_reg, y_pred)
print('MSE:{:.6f}'.format(mse))

# 分类问题
model = XGBClassifier(objective=log_cosh_obj)
model.fit(X_train_clf, y_train_clf)
y_pred, y_prob = model.predict(X_test_clf), model.predict_proba(X_test_clf)[:, -1]
auc = roc_auc_score(y_test_clf, y_prob)
f1 = f1_score(y_test_clf, y_pred)
acc = accuracy_score(y_test_clf, y_pred)
print('AUC:{:.6f}, F1:{:.6f}, Acc:{:.6f}'.format(auc, f1, acc))

MSE:1133.472053
AUC:0.922540, F1:0.825397, Acc:0.824000


### 2. Pseudo-Huber loss function，可以近似替代MAE

In [3]:
# 定义损失函数
def huber_approx_obj(y_true, y_pred, h=1):
    # h为Pseudo-Huber loss function中的参数，用于调节坡度，其值越大，图像越陡峭
    d = y_pred - y_true 
    scale = 1 + np.square(d / h)
    scale_sqrt = np.sqrt(scale)
    grad = d / scale_sqrt
    hess = 1 / scale / scale_sqrt
    return grad, hess

# 回归问题
model = XGBRegressor(objective=huber_approx_obj)
# model = XGBRegressor(objective='reg:linear')
model.fit(X_train_reg, y_train_reg)
y_pred = model.predict(X_test_reg)
mse = mean_squared_error(y_test_reg, y_pred)
print('MSE:{:.6f}'.format(mse))

# 分类问题
model = XGBClassifier(objective=huber_approx_obj)
model.fit(X_train_clf, y_train_clf)
y_pred, y_prob = model.predict(X_test_clf), model.predict_proba(X_test_clf)[:, -1]
auc = roc_auc_score(y_test_clf, y_prob)
f1 = f1_score(y_test_clf, y_pred)
acc = accuracy_score(y_test_clf, y_pred)
print('AUC:{:.6f}, F1:{:.6f}, Acc:{:.6f}'.format(auc, f1, acc))

MSE:1111.974434
AUC:0.928238, F1:0.830040, Acc:0.828000


### 3. 以log(exp(-x) + exp(x))为损失函数：处理分类问题较合适

In [4]:
# 定义损失函数
def log_exp(y_true, y_pred):
    d = y_pred - y_true
    t1 = np.exp(d) - np.exp(-d) 
    t2 = np.exp(d) + np.exp(-d) 
    grad = t1 / t2
    hess = 1.0 - grad**2 
    return grad, hess

# 分类问题
model = XGBClassifier(objective=log_exp)
model.fit(X_train_clf, y_train_clf)
y_pred, y_prob = model.predict(X_test_clf), model.predict_proba(X_test_clf)[:, -1]
auc = roc_auc_score(y_test_clf, y_prob)
f1 = f1_score(y_test_clf, y_pred)
acc = accuracy_score(y_test_clf, y_pred)
print('AUC:{:.6f}, F1:{:.6f}, Acc:{:.6f}'.format(auc, f1, acc))

AUC:0.922540, F1:0.825397, Acc:0.824000


### 4. Fair Loss：意义不大

In [7]:
def fair_obj(y_true, y_pred, c=1):
    x = y_true - y_pred
    den = np.abs(x) + c
    grad = c*x / den
    hess = c*c / np.square(den) 
    return grad, hess

model = XGBRegressor(objective=fair_obj, n_estimators=500, learning_rate=0.15)
# model = XGBRegressor(objective='reg:linear')
model.fit(X_train_reg, y_train_reg)
y_pred = model.predict(X_test_reg)
mse = mean_squared_error(y_test_reg, y_pred)
print('MSE:{:.6f}'.format(mse))


model = XGBClassifier(objective=fair_obj, n_estimators=500, learning_rate=0.15)
model.fit(X_train_clf, y_train_clf)
y_pred, y_prob = model.predict(X_test_clf), model.predict_proba(X_test_clf)[:, -1]
auc = roc_auc_score(y_test_clf, y_prob)
f1 = f1_score(y_test_clf, y_pred)
acc = accuracy_score(y_test_clf, y_pred)
print('AUC:{:.6f}, F1:{:.6f}, Acc:{:.6f}'.format(auc, f1, acc))

MSE:2724600341.150338
AUC:0.208725, F1:0.263566, Acc:0.240000


### 4. 自定义函数：适合处理分类问题

In [5]:
def lodd(y_true, y_pred):
    d = y_pred - y_true
    grad = np.log1p(d) 
    hess = 0.5 / (abs(d)+0.001) 
    return grad, hess

model = XGBClassifier(objective=lodd)
model.fit(X_train_clf, y_train_clf)
y_pred, y_prob = model.predict(X_test_clf), model.predict_proba(X_test_clf)[:, -1]
auc = roc_auc_score(y_test_clf, y_prob)
f1 = f1_score(y_test_clf, y_pred)
acc = accuracy_score(y_test_clf, y_pred)
print('AUC:{:.6f}, F1:{:.6f}, Acc:{:.6f}'.format(auc, f1, acc))